In [1]:
import cv2
import mediapipe as mp
from math import hypot
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np

In [2]:
import cv2
import mediapipe as mp
from math import hypot
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np
 
cap = cv2.VideoCapture(0) 
 
mpHands = mp.solutions.hands 
hands = mpHands.Hands()   
mpDraw = mp.solutions.drawing_utils
 
 
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volbar=400
volper=0
 
volMin,volMax = volume.GetVolumeRange()[:2]
 
while True:
    success,img = cap.read() 
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) 
    
    
    results = hands.process(imgRGB) 
 
    lmList = [] 
    if results.multi_hand_landmarks: #list of all hands detected.
        #By accessing the list, we can get the information of each hand's corresponding flag bit
        for handlandmark in results.multi_hand_landmarks:
            for id,lm in enumerate(handlandmark.landmark): #adding counter and returning it
                # Get finger joint points
                h,w,_ = img.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                lmList.append([id,cx,cy]) #adding to the empty list 'lmList'
            mpDraw.draw_landmarks(img,handlandmark,mpHands.HAND_CONNECTIONS)
    
    if lmList != []:
        #getting the value at a point
                        #x      #y
        x1,y1 = lmList[4][1],lmList[4][2]  #thumb
        x2,y2 = lmList[8][1],lmList[8][2]  #index finger
        #creating circle at the tips of thumb and index finger
        cv2.circle(img,(x1,y1),13,(255,0,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.circle(img,(x2,y2),13,(255,0,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.line(img,(x1,y1),(x2,y2),(255,0,0),3)  #create a line b/w tips of index finger and thumb
 
        length = hypot(x2-x1,y2-y1) #distance b/w tips using hypotenuse
 # from numpy we find our length,by converting hand range in terms of volume range ie b/w -63.5 to 0
        vol = np.interp(length,[30,350],[volMin,volMax]) 
        volbar=np.interp(length,[30,350],[400,150])
        volper=np.interp(length,[30,350],[0,100])
        
        
        print(vol,int(length))
        volume.SetMasterVolumeLevel(vol, None)
        
        # Hand range 30 - 350
        # Volume range -63.5 - 0.0
        #creating volume bar for volume level 
        cv2.rectangle(img,(50,150),(85,400),(0,0,255),4) # vid ,initial position ,ending position ,rgb ,thickness
        cv2.rectangle(img,(50,int(volbar)),(85,400),(0,0,255),cv2.FILLED)
        cv2.putText(img,f"{int(volper)}%",(10,40),cv2.FONT_ITALIC,1,(0, 255, 98),3)
        #tell the volume percentage ,location,font of text,length,rgb color,thickness
    cv2.imshow('Image',img) #Show the video 
    if cv2.waitKey(1) & 0xff==ord(' '): #By using spacebar delay will stop
        break
        
cap.release()     #stop cam       
cv2.destroyAllWindows() #close window

-40.54292848461317 151
-62.716177973920836 42
-57.8954794748735 66
-63.80064444173759 37
-61.70581759291926 47
-58.96237854224816 60
-60.30915021490131 54
-59.82890410763274 56
-58.964054528063784 60
-57.3213174502629 68
-63.42530736472927 38
-65.25 20
-65.25 19
-61.25787836912881 49
-65.25 17
-65.25 25
-62.047534174347184 45
-65.25 24
-65.25 19
-65.25 27
-63.79789746649434 37
-63.2109375 40
-60.66259487708079 52
-60.75622285013045 52
-63.608025513898504 38
-61.20864442953693 49
-62.64915199500197 42
-64.19559858107344 35
-64.52796395944401 33
-62.98468117113179 41
-64.28896743130898 34
-60.146784048551325 55
-63.20074855161765 40
-65.25 27
-65.25 20
-61.59456697434381 47
-64.08056976790833 35
-62.56372554558613 43
-65.25 27
-65.25 14
-64.51278253445845 33
-61.93886589480347 46
-62.047534174347184 45
-65.25 17
-65.25 17
-58.41312117036218 63
-63.98138899670667 36
-57.16676038927713 69
-65.25 25
-65.25 8
-65.25 29
-62.58973986209678 43
-65.25 29
-64.02373101484166 36
-65.25 29
-65.25 24